# Índice<a id="ind"></a>

[**Introdução**](#limp_ini)

[**Etapa 1**](#etapa1)
* [Análise de uma amostra: identificação tipos mínimos e transformações](#ana_amo)
        
[**Etapa 2**](#etapa2)
* [Carregamento e tratamento dos dados totais](#carr_tot)
* [Comparação entre dados com e sem optimização](#comp)

In [1]:
# instala o pacote "optimizacaoDF"
!pip install git+https://github.com/joaogambaro/optimizacao_dataframes

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/joaogambaro/optimizacao_dataframes to /tmp/pip-req-build-oz0mdxts
  Running command git clone --filter=blob:none --quiet https://github.com/joaogambaro/optimizacao_dataframes /tmp/pip-req-build-oz0mdxts
  Resolved https://github.com/joaogambaro/optimizacao_dataframes to commit 4aa7233977ba0e6952eee6480f5331c670cd4e68
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

import optimizacaoDF.coleta_amostra as amo
import optimizacaoDF.analise_resultados as ana
import optimizacaoDF.optimizacao_dataframe as opt

import os
import importlib
import warnings

pd.pandas.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

# Introdução<a id="limp_ini"></a>
[Índice](#ind)

O objetivo central deste notebook foi desenvolver uma rotina para abrir um arquivo ".csv" com um   consumo reduzido de memória.  A redução foi obtida identificando as colunas de dados que pode ser escritas com subtipos de variáveis menores.

    Exemplo: para os dados abaixo, a coluna "idade" foi carregada com o tipo `float64`, "nome" com o tipo `object` e "peso" com o tipo `float64`. Contudo, estes tipos necessitam de mais memória do que o necessário para o armazenamento dos dados em questão. Para estes dados,  por exemplo, poderíamos escrever as colunas com os tipo, int8 para "idade", "classify" para "nome" e float32 para "peso" sem perda de informação, reduzindo o uso de memória.

|idade(float64)|nome(object) | peso(float64)|
|--|--|--|
|34.0| João|64.5 |
|32.0| Camila|75.8 |
|31.0| José|65.3 |
|26.0| Ana|58.0 |
    
Este trabalho foi dividido em etapas

**Etapa 1: analisar um pequena amostra dos dados**

Nesta etapa é carregado em um dataframe com uma amostra dos dados completos. O dataframe contém somente algumas linhas do arquivo original, que podem ser escolhidas aleatoriamente ou em sequência. A amostra selecionada é usada para identificar os subtipos mínimos de cada coluna e as transformações que podem ser feitas.

**Etapa 2: carregamento dos dados totais**

Nesta etapa os dados são carregados utilizando o subtipos mínimos identificados na etapa 1. Depois do carregamento, as transformações identificadas na etapa 1 são aplicadas e, por final, uma nova identificação de subtipos é feita. Estes últimos passos  podem reduzir ainda mais o tamanho do dataframe  na memória. O resultado final é salvo em um novo arquivo.


**Observações**
- para auxiliar nos processo das etapa 1 e 2 foi criado o pacote "optimizacaoDF" que foi instalado no incio do notebook

- Quando os dados são salvos no final do processo, os subtipos que foram definidos não são salvos no arquivo. Ou seja, quando os dados forem abertos outras vezes, devemos novamente informar os tipos para reduzir o consumo de memória. Ainda assim, é importante salvar os o resultado final em um arquivo, pois muitos arquivos grandes possuem linhas em branco que serão eliminadas com o salvamento do resultado final. Além disso, as demais transformações feitas nos dados ficarão salvas com o novo arquivo.

# Etapa 1<a id="etapa1"></a>
[Índice](#ind)

## Análise de uma amostra: identificação tipos mínimos e transformações<a id="ana_amo"></a>
   

- Identificação dos subtipos mínimos: 

Este processo é feito usando o método `obtem_subtipos` do pacote `optimizacaoDF`, que retorna um dicionário com os nomes das colunas e os subtipos mínimos.

>Observações: 
>- neste trabalho não foi identifica nenhuma transformação necessária nos dados além das mudanças nos subtipos.
>- no método obtem_subtipos foi usado "can_float_be_int=False" para que colunas com float nunca sejam identificadas como int (se fosse usado True o número 23.00, por exemplo, poderia ser identificado como o int, 23). Como estamos lidando com uma amostra, a melhor opção é usar False para este parâmetro. Isto porque com True o método pode associar uma coluna float como int para a amostra, enquanto que nos dados totais pode ser que isto seja impossível. Este fato pode acontecer, por exemplo, quando quando na amostra não existem dados Nan (que são float), mas nos dados totais existem.

In [3]:
# caminho do arquivo com dados
current_path = os.getcwd()
file_name=Path(current_path)/"dados_originais/2_reviews.csv"

# coleta uma amostra(primeiras linhas)
tamAm=500
df=amo.coleta_amostr(tamAm, file_name, is_rand_sample=False)

print("df.shape: ",df.shape)
display(df.head(3))

df.shape:  (500, 6)


,listing_id,id,date,reviewer_id,reviewer_name,comments
0,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
1,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
2,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...


In [4]:
# obtem os subtipos minimos
dict_sub = opt.obtem_subtipos(df,can_float_be_int=True)
dict_sub

{'listing_id': 'uint32',
 'id': 'uint64',
 'date': 'object',
 'reviewer_id': 'uint32',
 'reviewer_name': 'object',
 'comments': 'object'}

# Etapa 2<a id="etapa2"></a>
[Índice](#ind)

## Carregamento e tratamento dos dados totais<a id="carr_tot"></a>

Rotina seguida:
- 1. Carrega os dados com os subtipos mínimos identificados com a amostra

- 2. Salva os dados em um arquivo

In [7]:
# carrega o dataframe com os tipos minimos
df_final=pd.read_csv(file_name,
                 low_memory=False,                 
                 dtype= dict_sub   #dicionário com subtipos minimos 
                )

# salva o arquivo
current_path = os.getcwd()
path_root=Path(current_path).parent
df_final.to_csv(path_root/"dados/2_reviews_opt.csv", index=False)

# Comparação entre dados com e sem optimização<a id="comp"></a>
[Índice](#ind)

Para finalizar, demostro a eficácia do método usado. Com este objetivo foram carregados os dados completos sem a identificação dos subtipos e foi medido o consumo de memória para os dados com e sem o tratamento proposto. Fazendo isto, podemos verificar que há uma redução de apenas 1,61% na alocação de memória para este arquivo, mas reduções mais expressivas foram observadas em outros casos (como no arquivo "1_listings.csv" onde a redução foi de 87.46%).

Também é mostrada uma comparação entre os tipos para os dados originais e para os dados com o tratamento. Esta comparação é feita pelo método `resultados_em_df` do pacote `optimizacaoDF`

>**Observação:** a ideia do tratamento é não precisar carregar os dados completos com o seu consumo máximo de memória. Para isto, utilizamos uma amostra para identificar os subtipos usados no carregamento dos dados totais. Nesta seção, os dados totais foram carregados na sua forma menos eficiente somente para mostrar a eficácia dos métodos aplicados, mas ideia central é não precisar fazer este carregamento.

In [6]:
# le o arquivo original(sem tranformação e sem conversao de tipo)
df_original = pd.read_csv(file_name,
                          low_memory=False,                          
                         )


# redução de momória
porc = ana.porc_reducao(df_original,df_final)*100
mem_df_original = ana.uso_memoria(df_original)
mem_df_final = ana.uso_memoria(df_final)

print(f'memória do df_original: {round(mem_df_original,2)}Mb')
print(f'memória do df_final: {round(mem_df_final,2)}Mb')
print(f'Porcentagem na redução de memória: {round(porc,2)}%')

# compara com o resultado da optimização
ana.resultados_em_df(df_original, df_final)

memória do df_original: 217.8Mb
memória do df_final: 214.3Mb
Porcentagem na redução de memória: 1.61%


,sem_mud,com_mud
listing_id,int64,uint32
id,int64,uint64
date,object,object
reviewer_id,int64,uint32
reviewer_name,object,object
comments,object,object
uso_mem_MB,217.798783,214.301171
